# Malicious Service Installations
Detects known malicious service installs that only appear in cases of lateral movement, credential dumping and other suspicious activity

## Rule Content
```
- title: Malicious Service Installations
  id: 5a105d34-05fc-401e-8553-272b45c1522d
  description: Detects known malicious service installs that only appear in cases
    of lateral movement, credential dumping and other suspicious activity
  author: Florian Roth
  tags:
  - attack.persistence
  - attack.privilege_escalation
  - attack.t1050
  - car.2013-09-005
  logsource:
    product: windows
    service: system
    category: null
  detection:
    selection:
      EventID: 7045
    malsvc_wce:
      ServiceName:
      - WCESERVICE
      - WCE SERVICE
    malsvc_paexec:
      ServiceFileName: '*\PAExec*'
    malsvc_winexe:
      ServiceFileName: winexesvc.exe*
    malsvc_pwdumpx:
      ServiceFileName: '*\DumpSvc.exe'
    malsvc_wannacry:
      ServiceName: mssecsvc2.0
    malsvc_persistence:
      ServiceFileName: '* net user *'
    malsvc_others:
      ServiceName:
      - pwdump*
      - gsecdump*
      - cachedump*
    condition: selection and 1 of malsvc_*
  falsepositives:
  - Penetration testing
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-system-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"7045" AND (service_name:("WCESERVICE" OR "WCE\ SERVICE") OR service_image_path.keyword:*\\PAExec* OR service_image_path.keyword:winexesvc.exe* OR service_image_path.keyword:*\\DumpSvc.exe OR service_name:"mssecsvc2.0" OR service_image_path.keyword:*\ net\ user\ * OR service_name.keyword:(pwdump* OR gsecdump* OR cachedump*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()